# 3. Alternative designs

This notebook challenges you to optimize the design of the Sand Motor to maximize dune growth.
In order to do so this notebook provides you with some Python functions to alter the model schematizations. In addition, along with this notebook a set of alternative input files is shipped that you can use to alter the initial Sand Motor model.
Note that these model schematizations are highly optimized for speed and only have an educational purpose.

Execute cells by selecting the cell and press **Shift-Enter**.

### Helper functions

Execute the next cell with **Shift-Enter** to enable the helper functions.
Adapt the functions if appropriate.

In [ ]:
%matplotlib inline

from plot_tools import *

## 3.1 Lower construction height

We've seen that low-lying beaches can have an important role in aeolian sediment supply when waves are present.
By altering the construction height of the Sand Motor you might be able to stimulate aeolian sediment supply.

The following Python function reads and writes bathymetries and allows you to lower a bathymetry above a given level with a given factor. You can use this function to explore the effect of alternative construction heights. Execute the next cell with **Shift-Enter** to enable the function.

In [ ]:
def alter_construction_height(z_in, z_out, level=0., factor=0.5):
    '''Reads a bathy, reduces it's height above `level` by `factor`.
    
    Parameters
    ----------
    z_in : str
      Input bed level file
    z_out : str
      Output bed level file
    level : float
      Bed level above which bathymetry should be lowered
    factor : float
      Factor with which the bed level above `level` should be loaded
      
    '''
    
    z = np.loadtxt(z_in)
    ix = z >= level
    z[ix] = level + (z[ix] - level) * factor
    np.savetxt(z_out, z)

The next cell creates a bathymetry where the maximum height of the Sand Motor is lowered from 7.5 m+MSL to 3 m+MSL:

In [ ]:
alter_construction_height('../data/bathy/coarse/z_20110803.txt',
                          '../data/bathy/lowered/z_20110803.txt', level=0, factor=0.4)

## Vegetation strategies

Vegetation *lowers* the local wind shear, resulting in a reduction in sediment transport capacity of the wind.
The effect of vegetation can be incorporated in the model by *increasing* the wind shear velocity threshold.
AeoLiS does not have a vegetation settlement and propagation module.
However, you can use masks to locally alter the wind shear velocity threshold.
A mask is a field with complex numbers where the real part of the complex number is multiplied by the instatnaneous velocity threshold and the complex part is added to the velocity threshold.
For example, a mask value of 2+0.5i will first multiply the instantaneous wind shear velocity threshold with two and than add 0.5 m/s.

The following Python function reads and writes threshold fields and allows you to increase the threshold above a given bed level with a given factor. You can use this function to explore the effect of vegetation strategies. Execute the next cell with **Shift-Enter** to enable the function.

In [ ]:
def create_threshold_file(z_in, th_in, th_out, level=0., mult=1.0, add=0.0):
    '''Creates a threshold mask for vegetation modulation based on a bathymetry
    
    Parameters
    ----------
    z_in : str
      Input bed level file
    th_in : str
      Input threshold file
    th_out : str
      Output threshold file
    level : float
      Bed level above which threshold should be increased
    mult : float
      Multiplication mask
    add : float
      Addition mask
      
    '''
    
    z = np.loadtxt(z_in)
    th = np.loadtxt(th_in, dtype=np.complex)
    ix = z >= level
    th[ix] += (z[ix] - level) * mult + add * 1j
    np.savetxt(th_out, th)

The next cell creates a threshold field where the threshold velocity at the Sand Motor above MSL is increased with a factor 2:

In [ ]:
create_threshold_file('../data/bathy/coarse/z_20110803.txt',
                      '../data/bathy/coarse/threshold.txt',
                      '../data/bathy/coarse/threshold_vegetated.txt', level=0, mult=2., add=0.)